## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 ./setup.sh 파일을 실행하거나 ./start_notebook.sh 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. milvus서버가 떠 있어야 정상 동작합니다. ./start_milvus.sh 를 실행하여 milvus 서버를 띄워주세요.

## 목적:
- LangGraph를 사용하여 Milvus에 벡터 데이터를 저장하고 검색
- 텍스트 데이터를 벡터로 변환하여 Milvus에 저장 후 유사 검색 수행

## 왜 필요한가?
- 멀티모달 데이터(텍스트, 이미지, 오디오 등) 검색을 효과적으로 수행할 수 있음
- LangGraph를 사용하면 데이터 처리를 노드 기반 워크플로우로 관리 가능

## 주요 개념:
- **Milvus**: 대규모 벡터 검색을 지원하는 오픈소스 벡터 DB
- **멀티모달 검색**: 텍스트, 이미지, 오디오 데이터를 동시에 검색하는 기술

In [1]:
# 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
import numpy as np
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import HuggingFaceEmbeddings
from pymilvus import connections, Collection, CollectionSchema, FieldSchema, DataType, utility

In [2]:
# Milvus 연결 설정
connections.connect("default", host="localhost", port="19530")

In [3]:
collection_name = "ex9_multimodal_vectors"

# Milvus 컬렉션 자동 생성 (없으면 생성)
if not utility.has_collection(collection_name):
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=384)
    ]
    schema = CollectionSchema(fields=fields)
    milvus_collection = Collection(name=collection_name, schema=schema)
    print(f"Milvus 컬렉션 '{collection_name}' 생성 완료!")
else:
    milvus_collection = Collection(name=collection_name)
    print(f"Milvus 컬렉션 '{collection_name}' 로드 완료!")

Milvus 컬렉션 'ex9_multimodal_vectors' 생성 완료!


In [4]:
# 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
# LangGraph에서 사용할 상태 타입 정의
class VectorState(TypedDict):
    text: str
    vector: Annotated[list[float], "vector"]

# 벡터 생성 함수
def generate_vector(state: VectorState) -> VectorState:
    vector = embeddings.embed_query(state["text"])
    if vector is None:
        raise ValueError(f"텍스트 '{state['text']}'에 대한 벡터를 생성할 수 없습니다.")
    print(f"'{state['text']}' → 벡터 변환 완료")
    return {"text": state["text"], "vector": vector}

# Milvus에 벡터 저장 및 인덱스 생성 함수
def store_in_milvus(state: VectorState) -> VectorState:
    if state["vector"] is None:
        raise ValueError("벡터 값이 None입니다. Milvus에 저장할 수 없습니다.")
    milvus_collection.insert([[state["vector"]]])
    print(f"'{state['text']}' 벡터가 Milvus에 저장됨")
    
    # 데이터 플러시
    milvus_collection.flush()
    print(f"데이터 플러시 완료")
    
    # 인덱스 생성
    index_params = {"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": 128}}
    milvus_collection.create_index(field_name="vector", index_params=index_params)
    print(f"인덱스 생성 완료")
    
    return state

# Milvus에서 벡터 검색 함수
def search_milvus(state: VectorState):
    if state["vector"] is None:
        raise ValueError("벡터 값이 None입니다. 검색을 수행할 수 없습니다.")
    
    # 컬렉션을 메모리에 로드
    milvus_collection.load()
    print(f"Milvus 컬렉션 '{collection_name}' 메모리에 로드 완료!")
    
    query_vector = np.array([state["vector"]])
    search_params = {"metric_type": "L2"}
    limit = 3  # 반환할 최대 결과 수
    results = milvus_collection.search(
        data=query_vector,
        anns_field="vector",
        param=search_params,
        limit=limit
    )
    print(f"'{state['text']}' 유사 벡터 검색 완료!")
    return {"results": results}

In [6]:
# StateGraph 그래프 정의
graph = StateGraph(VectorState)
graph.add_node("generate_vector", generate_vector)
graph.add_node("store_in_milvus", store_in_milvus)
graph.add_node("search_milvus", search_milvus)
graph.add_edge(START, "generate_vector")
graph.add_edge("generate_vector", "store_in_milvus")
graph.add_edge("store_in_milvus", "search_milvus")
graph.add_edge("search_milvus", END)

In [7]:
# 그래프 컴파일 및 실행
app = graph.compile()
app.invoke({"text": "Multimodal search with Milvus!"})

'Multimodal search with Milvus!' → 벡터 변환 완료
'Multimodal search with Milvus!' 벡터가 Milvus에 저장됨
데이터 플러시 완료
인덱스 생성 완료
Milvus 컬렉션 'ex9_multimodal_vectors' 메모리에 로드 완료!
'Multimodal search with Milvus!' 유사 벡터 검색 완료!


{'text': 'Multimodal search with Milvus!',
 'vector': [-0.019907398149371147,
  -0.04585188627243042,
  0.05553993582725525,
  -0.012397839687764645,
  0.04554487764835358,
  -0.016336878761649132,
  0.04541846364736557,
  -0.007620041258633137,
  -0.002043910324573517,
  -0.01930510625243187,
  0.00982975959777832,
  -0.058796051889657974,
  0.07074756920337677,
  0.04483018442988396,
  -0.05946190282702446,
  0.00893954187631607,
  -0.05293281748890877,
  0.15639503300189972,
  -0.006541709881275892,
  0.028106659650802612,
  0.026618923991918564,
  -0.031566206365823746,
  0.06439940631389618,
  -0.06151197478175163,
  -0.030508942902088165,
  0.051837291568517685,
  -0.016137605533003807,
  -0.05553052946925163,
  0.03181447461247444,
  -0.06607703864574432,
  0.025440320372581482,
  0.06426794826984406,
  0.055167533457279205,
  0.05767170339822769,
  0.009039425291121006,
  0.011478573083877563,
  -0.09166761487722397,
  -0.003853656817227602,
  -0.03222650662064552,
  0.01009946